In [1]:
from hamilton.plugins import h_tqdm
from hamilton import driver, base


In [ ]:
import aqs 

inputs = {
    'formatted_dir': r'D:\data\capstone_data\good_data\aqs\formatted',
    'unzip_dir': r'D:\data\capstone_data\good_data\aqs\unzips',
    'zip_dir': r'D:\data\capstone_data\good_data\aqs\zips'
}
outputs = ['total']

dr = (
    driver.Builder()
    .with_config({})
    .with_modules(aqs)
    .with_adapters(
        base.SimplePythonGraphAdapter(base.DictResult()),
        h_tqdm.ProgressBar(desc='AQS')
    )
    .build()
)

In [ ]:
dr.execute(final_vars=outputs, inputs=inputs);

In [ ]:
import os
import pandas as pd

aqs_path = r'D:\data\capstone_data\good_data\aqs\formatted'


datas = []
for file in os.listdir(aqs_path):
    data = pd.read_csv(os.path.join(aqs_path, file), index_col=0)
    chem, year = file.split('.csv')[0].split('_')
    data = data.groupby(['State Code', 'County Code'])['Observation Count'].sum().reset_index()
    data['Chemical'] = chem
    data['Year'] = year
    datas.append(data)

total = pd.concat(datas, ignore_index=True).pivot(index=['State Code', 'County Code', 'Year'], columns='Chemical', values='Observation Count').reset_index()
total.to_csv(r'D:\data\capstone_data\good_data\sets\aqs_data.csv', index=False)

In [ ]:
import tri 

inputs = {
    'download_dir': r'D:\data\capstone_data\good_data\tri',
}
outputs = ['csvs']

dr = (
    driver.Builder()
    .with_config({})
    .with_modules(tri)
    .with_adapters(
        base.SimplePythonGraphAdapter(base.DictResult()),
        h_tqdm.ProgressBar(desc='TRI')
    )
    .build()
)

In [ ]:
dr.execute(final_vars=outputs, inputs=inputs);

In [ ]:
datas = []
data_dir = r'D:\data\capstone_data\good_data\tri'
for file in os.listdir(data_dir):
    data = pd.read_csv(os.path.join(data_dir, file))
    grouped = data.groupby(['8. ST', '7. COUNTY', '1. YEAR', '37. CHEMICAL'])['107. TOTAL RELEASES'].sum().reset_index()
    datas.append(grouped.pivot(index=['8. ST', '7. COUNTY', '1. YEAR'], columns='37. CHEMICAL', values='107. TOTAL RELEASES').reset_index())

total = pd.concat(datas, ignore_index=True)
total.to_csv(r'D:\data\capstone_data\good_data\sets\tri_data.csv', index=False)

In [ ]:
import seers_populations

inputs = {
    'download_dir': r'D:\data\capstone_data\good_data\seers',
    'url': 'https://seer.cancer.gov/popdata/yr1969_2023.20ages/us.1969_2023.20ages.adjusted.txt.gz'
}
outputs = ['SEERSFormatted']

dr = (
    driver.Builder()
    .with_config({})
    .with_modules(seers_populations)
    .with_adapters(
        base.SimplePythonGraphAdapter(base.DictResult()),
        h_tqdm.ProgressBar(desc='Seers')
    )
    .build()
)

In [ ]:
res = dr.execute(final_vars=outputs, inputs=inputs);

In [ ]:
res['SEERSFormatted'].groupby(['STATE_FIPS_CODE', 'COUNTY_FIPS_CODE', 'YEARS'])['POPULATION'].sum().reset_index().to_csv(r'D:\data\capstone_data\good_data\sets\seers_data.csv', index=False)

In [4]:
import attains

inputs = {
    'unzip_path': r'D:\data\capstone_data\good_data\attains\unzip',
    'zip_path': r'D:\data\capstone_data\good_data\attains\attains.zip',
    'url': 'https://dmap-data-commons-ow.s3.amazonaws.com/data/attains/geospatial/gpkg/ATTAINS_Assessment_20250117_gpkg.zip'
}
outputs = ['attains_au_points']

dr = (
    driver.Builder()
    .with_config({})
    .with_modules(attains)
    .with_adapters(
        base.SimplePythonGraphAdapter(base.DictResult()),
        h_tqdm.ProgressBar(desc='Attains')
    )
    .build()
)

In [5]:
res = dr.execute(final_vars=outputs, inputs=inputs);

Attains -> attains_au_points:  67%|██████▋   | 2/3 [32:04<13:47, 827.20s/funcs] 
********************************************************************************
> attains_au_points [attains.attains_au_points()] encountered an error         <
> Node inputs:
{'unzipped_file': None}
********************************************************************************
Traceback (most recent call last):
  File "c:\Users\chris\AppData\Local\Programs\Python\Python313\Lib\site-packages\hamilton\execution\graph_functions.py", line 294, in execute_lifecycle_for_node
    result = __adapter.call_lifecycle_method_sync(
        "do_node_execute",
    ...<3 lines>...
        task_id=__task_id,
    )
  File "c:\Users\chris\AppData\Local\Programs\Python\Python313\Lib\site-packages\hamilton\lifecycle\base.py", line 1062, in call_lifecycle_method_sync
    return getattr(adapter, method_name)(**kwargs)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "c:\Users\chris\AppData\Local\Programs\Python\Pyth

DataSourceError: None: No such file or directory

In [ ]:
import pandas as pd

data = pd.read_csv(r"D:\data\capstone_data\good_data\Wonder_processed.csv")

In [12]:
data.groupby(['Diagnosis', 'State', 'Year'])['Number of Incidences'].sum().reset_index(name='Cases').to_csv(r'D:\data\capstone_data\good_data\sets\wonder_data.csv', index=False)

In [13]:
import tiger_county

inputs = {
    'unzip_path': r'D:\data\capstone_data\good_data\tiger\tiger_county',
    'zip_path': r'D:\data\capstone_data\good_data\tiger\tiger_county.zip',
    'county_url': 'https://www2.census.gov/geo/tiger/TIGER2024/COUNTY/tl_2024_us_county.zip'
}
outputs = ['unzipped_file']

dr = (
    driver.Builder()
    .with_config({})
    .with_modules(tiger_county)
    .with_adapters(
        base.SimplePythonGraphAdapter(base.DictResult()),
        h_tqdm.ProgressBar(desc='Tiger County')
    )
    .build()
)

In [14]:
res = dr.execute(final_vars=outputs, inputs=inputs);

Tiger County -> Execution Complete!: 100%|██████████| 2/2 [00:03<00:00,  1.68s/funcs]
